In [8]:
library('dplyr')


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [9]:
library('tidyr')

In [10]:
library('tibble')

In [11]:
library('DtD')

### Modelling

In [12]:
Bankrupts <- read.csv('C:/Users/pawel/PycharmProjects/Bankrupts_point_final.csv', fileEncoding="UTF-8-BOM")

In [30]:
# dropping first redundant column
Bankrupts$X<-NULL

In [31]:
# integer vector, which enables to identify given company observations
# length (grp) equals all observations in the sample
grp<- Bankrupts$grp

In [32]:
# integer scalar for the minimum number of observation required in each window
min_obs <- 63

In [33]:
# unique number of identifiers (companies) in the sample
grps <- unique(grp)

In [34]:
# matrix in which the results will be stored
out <- matrix(NA_real_, nrow = length(grps), ncol = 10, dimnames = list(NULL, c("mu", "vol", "n_iter", "success", "n_obs", "grp","vt","st","dd", "pd")))

In [35]:
# loop that enables to estimate the parameter (drift and asset's volatility) for each company
for(g in grps){    
    idx <- which(grps == g)
    keep <- which(grp %in% g)
    out[idx, c("n_obs", "grp")] <- c(length(keep), g)
    if(length(keep) < min_obs)
        next
    res <- with(
    Bankrupts[keep, ],
    BS_fit(S = S, D = dt, T. = T, r = RF_Rate, time = time, method = "mle"))
    out[idx, c("mu", "vol", "n_iter", "success")] <- rep(
    do.call(c, res[c("ests", "n_iter", "success")]), each = length(idx))
    }

In [36]:
out <- data.frame(out)

In [38]:
Results <- Bankrupts %>%
    arrange(grp, at) %>% 
    group_by(grp) %>% 
    summarise_all(last)

In [44]:
out <- merge(out, Results[,c('grp','dt')], by="grp")

In [45]:
# simulating observations for the next one-year period
for (g in grps){
    bs_sim <- BS_sim(vol = out[,'vol'][g], 
               mu = out[,'mu'][g], 
               dt = 1/out[,'n_obs'][g], 
               V_0 = tail(Bankrupts[Bankrupts['grp']==g,][,'at'],1), 
               D = out[,'dt'][g], 
               r = Bankrupts[Bankrupts['grp']==g,][,'RF_Rate'], 
               T. = 1)
    
    out[,'vt'][g] <- tail(bs_sim[,'V'],1)
    out[,'st'][g] <- tail(bs_sim[,'S'],1)
    }

In [46]:
# calculating DtD (Distance to Default)
out$dd <- (log(out$vt)-log(out$dt)+(out$mu-(0.5*(out$vol^2))))/out$vol

In [47]:
# calculating PD (Probability of Default)
out$pd<-pnorm(-out$dd)

In [48]:
head(out)

,grp,mu,vol,n_iter,success,n_obs,vt,st,dd,pd,dt
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0.01379056,0.1417186,19,1,252,10615.2662,4124.693341,3.4771658,2.533722e-04,6509.5000
2,2,-0.47679803,0.3515911,21,1,251,399.0195,218.558911,0.6687021,2.518428e-01,184.0660
3,3,0.09930781,0.2715854,16,1,252,161.3046,137.739289,7.1607186,4.012762e-13,24.5565
4,4,-0.11040273,0.1801733,27,1,253,497.7010,126.808829,0.5726888,2.834277e-01,395.5125
5,5,-0.20981372,0.2910424,27,1,252,110.9617,7.967843,-1.4606695,9.279469e-01,131.9120
6,6,-0.07397569,0.1799105,24,1,253,264.1419,50.837086,0.2822587,3.888726e-01,229.4180


In [49]:
mean(out$pd)

[1] 0.6050304